In [9]:
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

In [10]:
# Given Parameters
transmission_rates = [500, 1000, 1500, 2000, 2500, 3000]  # in kb/s
num_devices_list = [50, 100, 150, 200, 250, 300]
task_data = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # in Mbit
max_delay = 0.5  # in seconds
device_computing_capacity = [0.5, 1]  # in GHz
num_of_mecs = 10
mec_computing_capacity = 8  # in GHz
transmission_power = 4  # in Watts

# Constants
H_m = 1e-96  # Consumption factor of electricity
y_nt = 1e-6  # Weighted factor of local time cost
y_ne = 1e-12  # Weighted factor of local energy consumption
e_m = 1e-7  # Energy required to calculate a single bit of task data for MEC server
noise_power_spectral_density = 1e-9  # in Watts/Hz
channel_gain = 2e-10  # 
path_loss_index = 4  # Typical urban area path loss exponent
bandwidth = 1e9  # 1 MHz bandwidth
max_energy = 1000  # in Joules
clock_period = (800,1200)  # cycles per bit
penalty_factor = 10 ** -2.5

In [11]:
def fitness_function(x, task_data, transmission_power, bandwidth, noise_power_spectral_density, 
                     device_computing_capacity, mec_computing_capacity, apply_penalty=True):
    # Convert Mbit to bits for task sizes
    task_data_bits = task_data * 1e6  
    
    # Convert GHz to Hz for computing capacities
    device_capacity_hz = device_computing_capacity * 1e9  
    mec_capacity_hz = mec_computing_capacity * 1e9  / num_of_mecs
    
    # Local computation
    if x == 0:  
        # The amount of tasks executed locally (1 - P_k)
        P_k = 0
        local_task_data = task_data_bits * (1 - P_k)
        
        # Equation (1) Local time cost (TC_l)
        time_cost = local_task_data / device_capacity_hz
        
        # Equation (2) Local energy cost (EC_l)
        energy_cost = H_m * device_capacity_hz**2 * local_task_data
        
        # Equation (3) Weighted overhead (WC_l)
        total_cost = y_nt * time_cost + y_ne * energy_cost
        
    # MEC server computation
    else:  
        # Equation (4) for transmission rate Vu
        transmission_rates = bandwidth * np.log2(1 + (transmission_power * channel_gain) / 
                                 (noise_power_spectral_density * bandwidth))
        
        # Equation (5) for uplink transmission time overhead TCe1
        TCe1 = task_data_bits / transmission_rates
        
        # Assuming Ru is the mean of clock_period_range, which is the average CPU cycles required per bit
        Ru = np.random.uniform(*clock_period)
        
        
        # Equation (6) for task processing time at the MEC server TCe2
        mec_capacity_per_task = mec_computing_capacity / num_of_mecs
        TCe2 = (task_data_bits * Ru) / max(mec_capacity_per_task, task_data_bits / num_of_mecs)
        
        # Equation (7) for total time cost
        time_cost = TCe1 + TCe2
        
        # Equation (8) for processing energy cost on the MEC server
        energy_cost = e_m * task_data_bits
        
        # Total cost considering both energy and time
        total_cost = y_nt * time_cost + y_ne * energy_cost
    
    # Apply a penalty if the energy cost exceeds the maximum allowed and apply_penalty is True
    if energy_cost > max_energy and apply_penalty:
        total_cost += (energy_cost - max_energy) * penalty_factor  # Adjust the penalty factor as necessary
    
    return total_cost, time_cost, energy_cost


In [12]:
class QPSO:
    def __init__(self, num_devices, max_iter, task_data):
        self.num_devices = num_devices
        self.max_iter = max_iter
        self.task_data = np.array(task_data) * 1e6  # Convert Mbit to bits for task sizes
        self.X = np.random.randint(2, size=(num_devices, max_iter))  # Initial binary decisions for each device
        self.transmission_rate = transmission_rates
        
        # Initialize Personal best decisions and their fitness
        self.P = np.zeros((num_devices, 1), dtype=int)
        self.P_fitness = np.full(num_devices, np.inf)

        # Initialize global best decision and its fitness
        self.g = np.zeros(num_devices, dtype=int)  # Placeholder, will be updated in evaluate_initial_decisions_parallel
        self.g_fitness = np.inf  # Important: initialize g_fitness before evaluate_initial_decisions_parallel

        # Parameters for the probability threshold (to adapt eq. 16 and 17 for binary decisions)
        self.beta = 1.0  # A constant for controlling the step size towards global best
        self.alpha = 0.75  # A constant for controlling the convergence

        # Parallel evaluation of initial decisions to find personal bests
        self.evaluate_initial_decisions_parallel()

        # Initialize global best decision set
        self.g = self.X[:, 0]
        self.g_fitness = np.min(self.P_fitness)  # Use the best of the initial personal bests
        g_index = np.argmin(self.P_fitness)
        self.g = self.X[:, g_index]

    def evaluate_fitness_parallel(self, tasks):
        with ThreadPoolExecutor() as executor:
            # Create a mapping of future to (i, t) pair
            future_to_it = {}
            
            for i, (task_size, device_capacity, t) in enumerate(tasks):
                future = executor.submit(fitness_function, self.X[i, t], task_size, transmission_power,
                                        bandwidth, noise_power_spectral_density, device_capacity, mec_computing_capacity)
                future_to_it[future] = (i, t)

            for future in as_completed(future_to_it):
                i, t = future_to_it[future]
                total_cost, _, _ = future.result()  # Unpack the total_cost from the tuple

                # Update personal and potentially global best
                if total_cost < self.P_fitness[i]:
                    self.P[i, 0] = self.X[i, t]
                    self.P_fitness[i] = total_cost
                    if total_cost < self.g_fitness:
                        self.g = self.X[:, t]
                        self.g_fitness = total_cost

    def evaluate_initial_decisions_parallel(self):
        tasks = [(self.task_data[i % len(self.task_data)], device_computing_capacity[i % len(device_computing_capacity)], 0) for i in range(self.num_devices)]
        self.evaluate_fitness_parallel(tasks)

    def run(self):
        for t in range(self.max_iter):
            tasks = [(self.task_data[i % len(self.task_data)], device_computing_capacity[i % len(device_computing_capacity)], t) for i in range(self.num_devices)]
            self.evaluate_fitness_parallel(tasks)

            # Quantum-inspired position updates for next iteration
            for i in range(self.num_devices):
                if t < self.max_iter - 1:
                    phi = np.random.uniform(0, 1)

                    # Update the global best position probabilistically (adapting eq. 16 for binary)
                    u = np.random.rand()
                    threshold = self.beta * np.abs(self.g[i] - self.X[i, t]) * np.log(1/u)
                    if np.random.rand() < threshold:
                        self.g[i] = 1 - self.g[i]  # Flip the bit

                    # Update the particle position probabilistically (adapting eq. 17 for binary)
                    u_i = np.random.rand()
                    threshold = self.alpha * (self.g[i] - self.X[i, t]) * np.log(1/u_i)
                    if np.random.rand() < threshold:
                        self.X[i, t+1] = self.P[i, 0]
                    else:
                        self.X[i, t+1] = self.g[i]

        return self.g


In [13]:
def collect_performance_data():
    results = []
    # Iterate over the combinations of numbers of devices and task data sizes
    for num_devices in num_devices_list:
        for task_size in task_data:
            # Initialize the QPSO instance for each combination
            qpso = QPSO(num_devices, 100, [task_size])  # Pass task_size in a list to match expected format
            best_strategy = qpso.run()  # Run the optimization to get the best offloading strategy
            
            # Calculate performance metrics for the best strategy
            for device_capacity in device_computing_capacity:
                total_cost, time_cost, energy_cost = fitness_function(
                    best_strategy[0],
                    task_size, 
                    transmission_power,
                    bandwidth, 
                    noise_power_spectral_density, 
                    device_capacity, 
                    mec_computing_capacity
                )
                
                # Append the results for this combination
                results.append({
                    'Strategy': best_strategy,
                    'Number of Devices': num_devices,
                    'Task Size (Mbit)': task_size,
                    'Device Computing Capacity (GHz)': device_capacity,
                    'Total Cost': total_cost,
                    'Time Cost (s)': time_cost,
                    'Energy Cost (Joules)': energy_cost
                })

    # Convert the list of results into a pandas DataFrame for easier analysis and visualization
    return pd.DataFrame(results)

# Execute the function to collect the performance data
df_results = collect_performance_data()
df_results.head()


,Strategy,Number of Devices,Task Size (Mbit),Device Computing Capacity (GHz),Total Cost,Time Cost (s),Energy Cost (Joules)
0,"[0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",50,5,0.5,1.000000e-08,1.000000e-02,1.250000e-72
1,"[0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",50,5,1.0,5.000000e-09,5.000000e-03,5.000000e-72
2,"[1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, ...",50,10,0.5,8.673674e+00,8.673674e+06,1.000000e+00
3,"[1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, ...",50,10,1.0,8.674214e+00,8.674214e+06,1.000000e+00
4,"[0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, ...",50,15,0.5,3.000000e-08,3.000000e-02,3.750000e-72
